# Предобработка текстовых данных

Работа с текстовыми данными обычно начинается с предобработки. Грубо говоря, предобработка - это удаление всего лишнего и приведение к нужному формату. Предобработка может быть долгой и неприятной, но в большинстве случаев все сводится к использованию стандартных инструментов. Эта тетрадка познакомит вас с такими инструментами.

#### Установите все нужные библиотеки. Подробнее про каждую из них ниже

In [1]:
!pip install pymystem3
!pip install pymorphy2 # "pymorphy2[fast]" doesn't work on macOS arm
!pip install razdel
!pip install gensim
!pip install nltk
!pip install rusenttokenize
!pip install regex

  Using cached pymorphy2-0.9.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl.metadata (7.0 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)


In [2]:
# Скорее всего вы уже знакомы с библиотекой pymorphy2.
# pymorphy2[fast] - это оптимизированный pymorphy2, который работает точно также, но сильно быстрее
# Если у вас windows, то он вряд ли установится и вам придется пользоваться стандартным

### Регулярные выражения

Один из базовых, но в то же время самых полезных инструментов для предобработки текста - регулярные выражения. В вводной части курса им был посвящен целый семинар - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/first_module_intro/01_regular_expressions.ipynb  

Если вы не ходили подготовительную часть и чувствуете себя неуверенно при работе с регулярками - пройдитесь по семинару. Решить домашку тоже не помешает. В семинаре я буду предполагать, что вы уже знакомы с вещами, которые разобраны в подготовительной части. Но новые вещи я буду объяснять дополнительно!

## Разбиение на предложения, токенизация, нормализация

За каждым из этих трех терминов стоит большая и сложная подзадача NLP. Однако для каждой есть готовые решения, которые очень хорошо работают. 

### Разбиение на предложения

In [3]:
text = """Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE. Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.

Во-первых, NLI можно использовать для контроля качества генеративных моделей. Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод. Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя". С помощью модели NLI можно проверять, что из X следует Y (то есть в новом тексте нету "отсебятины", придуманной моделью), и что из Y следует X (т.е. вся информация, присутствовавшая в исходном тексте, в новом также отражена).

Во-вторых, с помощью моделей NLI можно находить нетривиальные парафразы и в целом определять смысловую близость текстов. Для русского языка уже существует ряд моделей и датасетов по перефразированию, но кажется, что можно сделать ещё больше и лучше. В статье Improving Paraphrase Detection with the Adversarial Paraphrasing Task предложили считать парафразами такую пару предложений, в которой каждое логически следует из другого – и это весьма логично. Поэтому модели NLI можно использовать и для сбора обучающего корпуса парафраз (и не-парафраз, если стоит задача их детекции), и для фильтрации моделей, генерирующих парафразы.

"""
# текст отсюда - https://habr.com/ru/post/582620/

В __nltk__ есть уже готовая функция для разбивки на предложения. 

In [53]:
from nltk import download as nltk_download
nltk_download('punkt')
nltk_download('stopwords')

[nltk_data] Downloading package punkt to /Users/a.kerimov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/a.kerimov/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /Users/a.kerimov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
sent_tokenize(text, 'russian')[:10]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".',
 'С помощью модели NLI можно проверять, что из X следует Y (то есть

Nltk также позволяет обучить свой токенизатор предложений под определенный корпус. Это делается не очень просто, но вот тут есть исчерпывающий туториал - https://nlpforhackers.io/splitting-text-into-sentences/

В __gensim__ тоже есть готовая функция

In [6]:
from gensim.summarization.textcleaner import split_sentences

ModuleNotFoundError: No module named 'gensim.summarization'

In [120]:
# это ещё и генератор, т.е. сразу подходит для больших корпусов
list(split_sentences(text))[:5]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".']

У DeepPavlov есть библиотека [**rusenttokenizer**](https://github.com/deepmipt/ru_sentence_tokenizer).

In [7]:
from rusenttokenize import ru_sent_tokenize

In [8]:
ru_sent_tokenize(text)[:10]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".',
 'С помощью модели NLI можно проверять, что из X следует Y (то есть

В проекте Natasha есть библиотека [**razdel**](https://github.com/natasha/razdel). Она чуть более навороченная.

In [9]:
from razdel import sentenize

In [10]:
sents = list(sentenize(text))

In [11]:
# числа тут - это спаны, индексы начала и конца предложения в изначальном тексте
sents[:5]

[Substring(0,
           244,
           'Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.'),
 Substring(245,
           322,
           'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.'),
 Substring(324,
           401,
           'Во-первых, NLI можно использовать для контроля качества генеративных моделей.'),
 Substring(402,
           635,
           'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.'),
 Substring(636,
           913,
           'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачам

In [12]:
# у объекта Substring есть атрибуты start, stop и text. С помощью них можно вытащить нужное
[sent.text for sent in sents[:5]]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".']

Если все-таки нужно добавить каких-то специфичных правил разбиения на предложения, можно опять же воспользоваться регулярными выражениями. Однако в этом случае регулярка будет посложнее. 

Давайте посмотрим, что получится, если в качестве разделителя использовать !?. пробел и заглавную букву.  

In [13]:
import re

In [14]:
re.split(r'[!?\.] [А-Я]', text)[:10]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE',
 'роме этого, модели NLI обладают прикладной ценностью по нескольким причинам.\n\nВо-первых, NLI можно использовать для контроля качества генеративных моделей',
 'сть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод',
 'овременные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя"',
 ' помощью модели NLI можно проверять, что из X следует Y (то есть в новом 

Проблема в том, что сам разделитель удаляется тоже, а нам нужно удалить только пробел между знаками препинания и заглавной буквой.

Решается эта проблема с помощью __look ahead__ и __look behind__ (название функционала в регулярных выражениях).  
Синтаксис там такой:  
 **(?<=pattern)** положительное look-behind условие  
 **(?<!pattern)** отрицательное look-behind условие   
 **(?=pattern)** положительное look-ahead условие   
 **(?!pattern)** отрицательное look-ahead условие   

Подробно про это написано тут: https://www.regular-expressions.info/lookaround.html  

Look behind и look ahead превращают паттерн в условный, то есть проверяется есть ли он (до или после, соответственно), но его захвата не происходит. 

Обернём наше регулярное выражение и посмотрим, что получается:

In [15]:
re.split(r'(?<=[\.?!]) +(?=[А-ЯЁ])', text.replace('\n', ' '))[:10]

['Задача NLI важна для компьютерных лингвистов, ибо она позволяет детально рассмотреть, какие языковые явления данная модель понимает хорошо, а на каких – "плывёт"; по этому принципу устроены диагностические датасеты SuperGLUE и RussianSuperGLUE.',
 'Кроме этого, модели NLI обладают прикладной ценностью по нескольким причинам.',
 'Во-первых, NLI можно использовать для контроля качества генеративных моделей.',
 'Есть масса задач, где на основе текста X нужно сгенерировать близкий к нему по смыслу текст Y: суммаризация, упрощение текстов, перефразирование, перенос стиля на текстах, текстовые вопросно-ответные системы, и даже машинный перевод.',
 'Современные seq2seq нейросети типа T5 (которая в этом году появилась и для русского языка) в целом неплохо справляются с такими задачами, но время от времени лажают, упуская какую-то важную информацию из Х, или, наоборот, дописывая в текст Y что-то нафантазированное "от себя".',
 'С помощью модели NLI можно проверять, что из X следует Y (то есть

## Токенизация

Мы разбили текст на предложения. Теперь предложения нужно разбить на токены. Под токенами обычно понимаются слова, но это могут быть и какие-то более длинные или короткие куски. 

Самый простой способ токенизации -- стандартный питоновский __str.split__ метод.  
По умолчанию он разбивает текст по последовательностям пробелов 

In [16]:
'1  2 3'.split(' ') # NB! .split() и .split(' ') - не одно и тоже

['1', '', '2', '3']

In [17]:
'1  2 3'.split()

['1', '2', '3']

In [18]:
text.split()[10:30]

['рассмотреть,',
 'какие',
 'языковые',
 'явления',
 'данная',
 'модель',
 'понимает',
 'хорошо,',
 'а',
 'на',
 'каких',
 '–',
 '"плывёт";',
 'по',
 'этому',
 'принципу',
 'устроены',
 'диагностические',
 'датасеты',
 'SuperGLUE']

Большая часть слов отделяется, но знаки препинания лепятся к словам.
Можно пройтись по всем словам и убрать из них пунктуацию с методом str.strip.

In [19]:
#основные знаки преминания хранятся в питоновском модуле string в punctuation
import string

In [20]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
# в этом списке не хватает кавычек-ёлочек, лапок, длинного тире и многоточия
string.punctuation += '«»—…“”'

In [22]:
[word.strip(string.punctuation) for word in text.split()][10:30]

['рассмотреть',
 'какие',
 'языковые',
 'явления',
 'данная',
 'модель',
 'понимает',
 'хорошо',
 'а',
 'на',
 'каких',
 '–',
 'плывёт',
 'по',
 'этому',
 'принципу',
 'устроены',
 'диагностические',
 'датасеты',
 'SuperGLUE']

Так не будут удаляться дефисы и точки в сокращениях, не разделенных пробелом.

In [23]:
'как-нибудь'.strip(string.punctuation)

'как-нибудь'

In [24]:
'т.е.'.strip(string.punctuation)

'т.е'

Такой способ токенизации может работать быстрее других из-за того, что используются только дефолтные инструменты питона. Если важно качество, то лучше пользоваться готовыми токенизаторами

Например, готовые токенизаторы есть в nltk. Они не удаляют пунктуацию, а выделяют её отдельным токеном.

**wordpunct_tokenizer** разбирает по регулярке - *'\w+|[^\w\s]+'* (попробуйте понять как она работает просто глядя на паттерн)

In [25]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize

In [26]:
wordpunct_tokenize(text)[:10]

['Задача',
 'NLI',
 'важна',
 'для',
 'компьютерных',
 'лингвистов',
 ',',
 'ибо',
 'она',
 'позволяет']

**word_tokenize** также построен на регулярках, но они там более сложные (учитывается последовательность некоторых 
символов, символы начала, конца слова и т.д). 

Специально подобранного под русский язык токенизатора там нет, 
но и с английским всё работает достаточно хорошо --
сокращения типа т.к собираются в один токен, дефисные слова тоже не разделяются, многоточия тут тоже не отделяются, но это можно поправить.

In [27]:
word_tokenize(text)[130:150]

['лажают',
 ',',
 'упуская',
 'какую-то',
 'важную',
 'информацию',
 'из',
 'Х',
 ',',
 'или',
 ',',
 'наоборот',
 ',',
 'дописывая',
 'в',
 'текст',
 'Y',
 'что-то',
 'нафантазированное',
 '``']

В генсиме тоже есть функция для токенизации

In [28]:
from gensim.utils import tokenize

In [29]:
# опять же, это генератор
list(tokenize(text, lowercase=True))[30:50]

['russiansuperglue',
 'кроме',
 'этого',
 'модели',
 'nli',
 'обладают',
 'прикладной',
 'ценностью',
 'по',
 'нескольким',
 'причинам',
 'во',
 'первых',
 'nli',
 'можно',
 'использовать',
 'для',
 'контроля',
 'качества',
 'генеративных']

И в razdel тоже есть токенизация

In [30]:
from razdel import tokenize as razdel_tokenize

In [31]:
list(razdel_tokenize(text))[:10]

[Substring(0, 6, 'Задача'),
 Substring(7, 10, 'NLI'),
 Substring(11, 16, 'важна'),
 Substring(17, 20, 'для'),
 Substring(21, 33, 'компьютерных'),
 Substring(34, 44, 'лингвистов'),
 Substring(44, 45, ','),
 Substring(46, 49, 'ибо'),
 Substring(50, 53, 'она'),
 Substring(54, 63, 'позволяет')]

In [32]:
[token.text for token in list(razdel_tokenize(text))[:10]]

['Задача',
 'NLI',
 'важна',
 'для',
 'компьютерных',
 'лингвистов',
 ',',
 'ибо',
 'она',
 'позволяет']

Работать с регистром тяжело и поэтому можно привести все к нижнему регистру

In [33]:
[token.text.lower() for token in list(razdel_tokenize(text))[:10]]

['задача',
 'nli',
 'важна',
 'для',
 'компьютерных',
 'лингвистов',
 ',',
 'ибо',
 'она',
 'позволяет']

# Нормализация

В последнее время нормализация (т.е. приведение токенов к стандартному виду) используется все реже. Это связано с использованием subword или byte токенизации в топовых моделях (подробнее об этом мы поговорим когда дойдем до нейронных сетей). Однако у них есть свои недостатки и забывать про нормализацию пока не стоит.

Два основных вида нормализации - лемматизация и стемминг. Стемминг уже нигде не используется, но его полезно разобрать, чтобы понимать почему нужно использовать лемматизацию (еще стемминг по непонятной причине часто упоминается как важный скилл в вакансиях)

## Стемминг

Стемминг - это урезание слова до его "основы" (стема), т.е. такой части, которая является общей для всех словоформ в парадигме слова *(Значения слов "слово", "слоформа", "парадигма" приблизительно соответствует тому, которое использует Зализняк вот тут - http://inslav.ru/images/stories/pdf/2002_Zalizniak_RIS_i_statji.pdf (стр. 21-22)) Но это на самом деле не важно)*. 

По крайней мере так в теории. На практике стемминг сводится к отбрасыванию частотных окончаний.

Самый известный стеммер - стеммер Портера (или snowball стеммер). 
Подробнее про стеммер Портера можно почитать вот тут - <https://medium.com/@eigenein/стеммер-портера-для-русского-языка-d41c38b2d340>  
А совсем подробнее вот тут - <http://snowball.tartarus.org/algorithms/russian/stemmer.html>  
Почему он так называется? Так назывался язык программирования, который Портер написал для стеммеров. Язык так называется в созвучие языку SNOBOL. Вот комментарий самого Портера:

`Since it effectively provides a ‘suffix STRIPPER GRAMmar’, I had toyed with the idea of calling it ‘strippergram’, but good sense has prevailed, and so it is ‘Snowball’ named as a tribute to SNOBOL, the excellent string handling language of Messrs Farber, Griswold, Poage and Polonsky from the 1960s.`

Готовые стеммеры для разных языков есть в nltk. Работают они вот так:

In [34]:
from nltk.stem.snowball import SnowballStemmer

In [35]:
stemmer = SnowballStemmer('russian')

In [36]:
[(word, stemmer.stem(word)) for word in word_tokenize(text)][:30]

[('Задача', 'задач'),
 ('NLI', 'NLI'),
 ('важна', 'важн'),
 ('для', 'для'),
 ('компьютерных', 'компьютерн'),
 ('лингвистов', 'лингвист'),
 (',', ','),
 ('ибо', 'иб'),
 ('она', 'он'),
 ('позволяет', 'позволя'),
 ('детально', 'детальн'),
 ('рассмотреть', 'рассмотрет'),
 (',', ','),
 ('какие', 'как'),
 ('языковые', 'языков'),
 ('явления', 'явлен'),
 ('данная', 'дан'),
 ('модель', 'модел'),
 ('понимает', 'понима'),
 ('хорошо', 'хорош'),
 (',', ','),
 ('а', 'а'),
 ('на', 'на'),
 ('каких', 'как'),
 ('–', '–'),
 ('``', '``'),
 ('плывёт', 'плывет'),
 ("''", "''"),
 (';', ';'),
 ('по', 'по')]

Недостатки стемминга достаточно очевидные:  
1) с супплетивными формами или редкими окончаниями слова стемминг работать не умеет  
2) к одной основе могут приводится разные слова  
3) к разным основам могут сводиться формы одного слова  
4) приставки не отбрасываются

# Лемматизация

Лемматизация - это замена словоформы слова в парадигме на какую-то заранее выбранную стадартную форму (лемму). 



Например, для разных форм глагола леммой обычно является неопределенная форма (инфинитив), а для существительного форма мужского рода единственного числа. Это позволяет избавиться от недостатков стемминга (будет, был - одна лемма), (пролить, пролом - разные). Однако лемматизация значительно сложнее. 

К счастью есть готовые хорошие лемматизаторы. Для русского основых варианта два: Mystem и Pymorphy.


### Mystem

In [37]:
from pymystem3 import Mystem
import os, json
mystem = Mystem()


Майстем работает немного лучше и сам токенизирует,
поэтому можно в него засовывать сырой текст.

In [38]:
# mystem.lemmatize функция лемматизации в майстеме
# сам объект mystem нужно заранее инициализировать
mystem.lemmatize(text)[:10]

['задача', ' ', 'NLI', ' ', 'важный', ' ', 'для', ' ', 'компьютерный', ' ']

In [39]:
# Если нужна грамматическая информация или надо сохранить ненормализованный текст,
# есть функция mystem.analyze
words_analized = mystem.analyze(text)

In [40]:
# возвращает она список словарей
# каждый словарь имеет либо одно поле 'text' (когда попался пробел) или text и analysis
# в analysis снова список словарей с вариантами разбора (первый самый вероятный)
# поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
# analysis - может быть пустым списком
words_analized[:10]

[{'analysis': [{'lex': 'задача', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'Задача'},
 {'text': ' '},
 {'analysis': [], 'text': 'NLI'},
 {'text': ' '},
 {'analysis': [{'lex': 'важный', 'wt': 1, 'gr': 'A=ед,кр,жен'}],
  'text': 'важна'},
 {'text': ' '},
 {'analysis': [{'lex': 'для', 'wt': 1, 'gr': 'PR='}], 'text': 'для'},
 {'text': ' '},
 {'analysis': [{'lex': 'компьютерный',
    'wt': 1,
    'gr': 'A=(пр,мн,полн|вин,мн,полн,од|род,мн,полн)'}],
  'text': 'компьютерных'},
 {'text': ' '}]

In [41]:
print('Слово - ', words_analized[0]['text'])
print('Разбор слова - ', words_analized[0]['analysis'][0])
print('Лемма слова - ', words_analized[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', words_analized[0]['analysis'][0]['gr'])

Слово -  Задача
Разбор слова -  {'lex': 'задача', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}
Лемма слова -  задача
Грамматическая информация слова -  S,жен,неод=им,ед


In [42]:
#леммы можно достать в одну строчку
[parse['analysis'][0]['lex'] for parse in words_analized if parse.get('analysis')][:10]

['задача',
 'важный',
 'для',
 'компьютерный',
 'лингвист',
 'ибо',
 'она',
 'позволять',
 'детально',
 'рассматривать']

Mystem умеет разбивать текст на предложения, но через питоновский интерфейс это сделать не получится. Нужно скачать mystem отсюда - https://yandex.ru/dev/mystem/

После этого сохранить текст в файл.

In [43]:
f = open('text.txt', 'w')
f.write(text)
f.close()

Из командной строки или из питона запустить майстем на нашем файле

In [44]:
# про параметры почитайте в !mystem -h
!/Users/a.kerimov/.local/bin/mystem -isc --format json text.txt text_parsed.txt

В целевом файле теперь лежит разобранный текст в jsonlines (json на каждой строчке)

In [45]:
import json
t = [json.loads(line) for line in open('text_parsed.txt')]

In [46]:
t[0][:10]

[{'analysis': [{'lex': 'задача', 'gr': 'S,жен,неод=им,ед'}], 'text': 'Задача'},
 {'text': ' '},
 {'analysis': [], 'text': 'NLI'},
 {'text': ' '},
 {'analysis': [{'lex': 'важный', 'gr': 'A=ед,кр,жен'}], 'text': 'важна'},
 {'text': ' '},
 {'analysis': [{'lex': 'для', 'gr': 'PR='}], 'text': 'для'},
 {'text': ' '},
 {'analysis': [{'lex': 'компьютерный', 'gr': 'A=пр,мн,полн'},
   {'lex': 'компьютерный', 'gr': 'A=вин,мн,полн,од'},
   {'lex': 'компьютерный', 'gr': 'A=род,мн,полн'}],
  'text': 'компьютерных'},
 {'text': ' '}]

Каждый объект в этом списке - параграф. Каждый параграф на предложения можно разбив по тегу '//s'

Ещё так вызывать майстем может понадобиться, если важна скорость.

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

Важным достоинством Mystem является то, что он работает не с отдельными словами, а с целым предложением. При определении нужной леммы учитывается контекст, что позволяет во многих случаях разрешать омонимию.

### Pymorphy

Pymorphy - открытый и развивается (но не очень активно, т.к. это все сложно)

Ссылка на репозиторий: https://github.com/kmike/pymorphy2

Попробуйте сразу установить быструю версию (pip install pymorphy2[fast])

У него нет втстроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью. 

In [47]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [48]:
# основная функция - pymorphy.parse
words_analized = [morph.parse(token) for token in word_tokenize(text)]

In [49]:
morph.parse("печь")

[Parse(word='печь', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='печь', score=0.571428, methods_stack=((DictionaryAnalyzer(), 'печь', 2223, 0),)),
 Parse(word='печь', tag=OpencorporaTag('INFN,impf,tran'), normal_form='печь', score=0.285714, methods_stack=((DictionaryAnalyzer(), 'печь', 2456, 0),)),
 Parse(word='печь', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='печь', score=0.142857, methods_stack=((DictionaryAnalyzer(), 'печь', 2223, 3),))]

In [50]:
# Она похожа на analyze в майстеме только возрващает список объектов Parse
# Первый в списке - самый вероятный разбор (у каждого есть score)
# Информация достается через атрибут (Parse.word - например)
# Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
# части речи или другие категории
print('Первое слово - ', words_analized[0][0].word)
print('Разбор первого слова - ', words_analized[0][0])
print('Лемма первого слова - ', words_analized[0][0].normal_form)
print('Грамматическая информация первого слова - ', words_analized[0][0].tag)
print('Часть речи первого слова - ', words_analized[0][0].tag.POS)
print('Род первого слова - ', words_analized[0][0].tag.gender)
print('Число первого слова - ', words_analized[0][0].tag.number)
print('Падеж первого слова - ', words_analized[0][0].tag.case)

Первое слово -  задача
Разбор первого слова -  Parse(word='задача', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='задача', score=1.0, methods_stack=((DictionaryAnalyzer(), 'задача', 94, 0),))
Лемма первого слова -  задача
Грамматическая информация первого слова -  NOUN,inan,femn sing,nomn
Часть речи первого слова -  NOUN
Род первого слова -  femn
Число первого слова -  sing
Падеж первого слова -  nomn


## Дополнительная очистка текста

Можно убрать стоп-слова (предлоги, союзы, местоимения, частотные слова). Сам термин стоп-слово происходит из информационного поиска, первый раз его упомянул [Питер Лун](https://en.wikipedia.org/wiki/Hans_Peter_Luhn) в 1959.  
Удаление таких слов позволяло сократить размер индекса и не сильно испортить выдачу или даже улучшить её, поднимая релевантность документам со значимыми словами. Со временем от такой практики, в основном, отказались - память стала дешевой (и повились всякие алгоритмы для сокращения потребления памяти), а для учёта значимости придумали TFIDF (про него на следующем занятии).  

Во многих поисковых движках стоп-слова всё ещё используются. Часто их используют и в практических задачах (классификации, тематическом моделировании). 

In [51]:
from nltk.corpus import stopwords

In [54]:
# стоп-слова есть в nltk
stops = stopwords.words('russian')
print(stops)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Список не идеальный и его можно расширять под свои задачи.

In [55]:
words_normalized = [morph.parse(token)[0].normal_form for token in word_tokenize(text)]
[word for word in words_normalized if word not in stops][:10]

['задача',
 'nli',
 'важный',
 'компьютерный',
 'лингвист',
 ',',
 'ибо',
 'позволять',
 'детально',
 'рассмотреть']

## Предобработка для других языков

Предобработка сильно зависит от языка. Полностью универсальнных токенизаторов, лемматизаторов не бывает, а рассказать о предобработке под все существующие языки не реально. Поэтому посмотрим дополнительно только на предобработку на английском языке. 

Nltk и gensim по умолчанию адаптированы под английский язык (а регулярки вообще не привязаны к языку), поэтому разбирать их еще раз не будем.

Библиотека, про которую стоит отдельно рассказать - [**SpaCy**](https://spacy.io/). Это многоцелевая многоязычная библиотека. Если вам понадобится серьезно работать с английским, то лучшим вариантом будет использовать SpaCy. Другие языки там тоже поддерживаются (см. документацию), но не настолько хорошо как английский язык. 

В SpaCy много всего и мы будем возвращаться к ней по ходу курса. Пока посмотрим на интрументы базовой предобработки.

In [56]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 966.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 2.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.2/395.2 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 4.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.6/789.6 kB 6.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.9 MB/s eta 0:00:0

In [57]:
# загружаем пайплайн для английского языка
import spacy

nlp = spacy.load("en_core_web_sm")


In [58]:

text = ("One of the most salient features of our culture is that it won't so much bullshit.” "
        "These are the opening words of the short book On Bullshit, written by the philosopher Harry Frankfurt. "
        "Fifteen years after the publication of this surprise bestseller, "
        "the rapid progress of research on artificial intelligence is forcing us to reconsider our conception "
        "of bullshit as a hallmark of human speech, with troubling implications. What do philosophical "
        "reflections on bullshit have to do with algorithms? As it turns out, quite a lot."
        )

In [59]:
doc = nlp(text)

Результат предобаботки очень похож на майстем - тут есть разбиение на предложения, на токены, лемматизация, определение части речи. Тэги тут используются другие, но при желании можно сделать более менее адекватный маппинг (возможно кто-то уже это сделал и нужно только погуглить)

In [60]:
for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)
    print()

One one NUM
of of ADP
the the DET
most most ADV
salient salient ADJ
features feature NOUN
of of ADP
our our PRON
culture culture NOUN
is be AUX
that that SCONJ
it it PRON
wo will AUX
n't not PART
so so ADV
much much ADJ
bullshit bullshit NOUN
. . PUNCT
” " PUNCT

These these PRON
are be AUX
the the DET
opening opening NOUN
words word NOUN
of of ADP
the the DET
short short ADJ
book book NOUN
On on ADP
Bullshit Bullshit PROPN
, , PUNCT
written write VERB
by by ADP
the the DET
philosopher philosopher NOUN
Harry Harry PROPN
Frankfurt Frankfurt PROPN
. . PUNCT

Fifteen fifteen NUM
years year NOUN
after after ADP
the the DET
publication publication NOUN
of of ADP
this this DET
surprise surprise NOUN
bestseller bestseller NOUN
, , PUNCT
the the DET
rapid rapid ADJ
progress progress NOUN
of of ADP
research research NOUN
on on ADP
artificial artificial ADJ
intelligence intelligence NOUN
is be AUX
forcing force VERB
us we PRON
to to PART
reconsider reconsider VERB
our our PRON
conception concept

В задачах вроде извлечения ключевых слов может пригодится вытащить из текста только noun phrases (по-русски это вроде называется именные группы существительного)

In [61]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['the most salient features', 'our culture', 'it', 'These', 'the opening words', 'the short book', 'Bullshit', 'the philosopher', 'Harry Frankfurt', 'the publication', 'this surprise bestseller', 'the rapid progress', 'research', 'artificial intelligence', 'us', 'our conception', 'bullshit', 'a hallmark', 'human speech', 'troubling implications', 'What', 'philosophical reflections', 'bullshit', 'algorithms', 'it']


In [62]:
# загружаем пайплайн для немецкого языка
import spacy

nlp = spacy.load("de_core_news_sm")


In [63]:

text = ("Vor den Stadien habe ich bis jetzt zum Glück noch keine wüsten Szenen gesehen."
        "Vorstandschef Timotheus Höttges habe sich ausgesprochen optimistisch gezeigt, schrieb Analyst Robert Grindle in einer Studie vom Montag. "
        "Während der dortigen Räterepublik war er nach dem Krieg in Künstlergruppen und Ausschüssen aktiv."
        "Welches Ergebnis die Diskussion auf EU-Ebene auch letztlich bringt, wichtig ist, dass die Preisentwicklung für die"
        "Menschen verträglicher gestaltet wird“, so Gusenbauer."
        "Weitere Informationen unter www.schnippenburg.de sowie www.eisenzeithaus.de. Es gibt neue Nachrichten auf noz.de!" 
        "Jetzt die Startseite neu laden."
        "Der Initiative 'Zivilcourage', die sich jahrelang für das Denkmal in Form eines offenen " 
        "Der islamistischen Szene Thüringens wurden nach Angaben des Thüringer Innenministeriums "
        "zuletzt etwa 125 Personen zugerechnet, der salafistischen Szene etwa 75 Personen."
        "Allerdings bestand er die EMV-Prüfung nicht, weil er Radios und DVB-T-Empfänger stört."
        )

In [64]:
doc = nlp(text)

In [65]:
for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)
    print()

Vor vor ADP
den der DET
Stadien Stadien NOUN
habe haben AUX
ich ich PRON
bis bis ADP
jetzt jetzt ADV
zum zu ADP
Glück Glück NOUN
noch noch ADV
keine kein DET
wüsten wüst ADJ
Szenen Szene NOUN
gesehen sehen VERB
. -- PUNCT

Vorstandschef Vorstandschef NOUN
Timotheus Timotheus PROPN
Höttges Höttges PROPN
habe haben AUX
sich sich PRON
ausgesprochen ausgesprochen ADV
optimistisch optimistisch ADV
gezeigt zeigen VERB
, -- PUNCT
schrieb schreiben VERB
Analyst Analyst NOUN
Robert Robert PROPN
Grindle Grindle PROPN
in in ADP
einer ein DET
Studie Studie NOUN
vom von ADP
Montag Montag NOUN
. -- PUNCT

Während während ADP
der der DET
dortigen dortig ADJ
Räterepublik Räterepublik NOUN
war sein AUX
er er PRON
nach nach ADP
dem der DET
Krieg Krieg NOUN
in in ADP
Künstlergruppen Künstlergruppe NOUN
und und CCONJ
Ausschüssen Ausschuß NOUN
aktiv aktiv ADV
. -- PUNCT

Welches welcher DET
Ergebnis Ergebnis NOUN
die der DET
Diskussion Diskussion NOUN
auf auf ADP
EU-Ebene EU-Eben NOUN
auch auch ADV
letztli

In [66]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['den Stadien', 'ich', 'Glück', 'noch keine wüsten Szenen', 'Vorstandschef Timotheus Höttges', 'sich', 'Analyst Robert Grindle', 'einer Studie', 'Montag', 'der dortigen Räterepublik', 'er', 'dem Krieg', 'Künstlergruppen', 'Ausschüssen', 'Welches Ergebnis', 'die Diskussion', 'EU-Ebene', 'die Preisentwicklung', 'dieMenschen', 'Weitere Informationen', 'www.schnippenburg.de', 'www.eisenzeithaus.de', 'neue Nachrichten', 'noz.de', 'die Startseite', "Der Initiative 'Zivilcourage", 'die', 'sich', 'das Denkmal', 'Form', 'Der islamistischen Szene', 'Thüringens', 'Angaben', 'des Thüringer Innenministeriums', 'etwa 125 Personen', 'der salafistischen Szene', 'etwa 75 Personen', 'er', 'die EMV-Prüfung', 'er', 'Radios']


Поддержка русского языка в spacy тоже не так давно добавилась, но она не полная

In [67]:
nlp = spacy.load("ru_core_news_sm")

In [68]:
# возьмем любой текст
text = "ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО "\
       "ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ "\
       "ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, "\
       "ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА"

In [70]:
doc = nlp(text)

for sent in doc.sents: # достаем предложения
    for token in sent: # достаем токены
        print(token.text, token.lemma_, token.pos_)

ДАННОЕ данное NOUN
СООБЩЕНИЕ сообщение PROPN
( ( PUNCT
МАТЕРИАЛ материал PROPN
) ) PUNCT
СОЗДАНО создано PROPN
И и PROPN
( ( PUNCT
ИЛИ или PROPN
) ) PUNCT
РАСПРОСТРАНЕНО распространено PROPN
ИНОСТРАННЫМ иностранным PROPN
СРЕДСТВОМ средством PROPN
МАССОВОЙ массовой PROPN
ИНФОРМАЦИИ информации PROPN
, , PUNCT
ВЫПОЛНЯЮЩИМ выполняющим PROPN
ФУНКЦИИ функция PROPN
ИНОСТРАННОГО иностранного PROPN
АГЕНТА агента PROPN
, , PUNCT
И и CCONJ
( ( PUNCT
ИЛИ или PROPN
) ) PUNCT
РОССИЙСКИМ российским PROPN
ЮРИДИЧЕСКИМ юридическим PROPN
ЛИЦОМ лицом PROPN
, , PUNCT
ВЫПОЛНЯЮЩИМ выполняющим PROPN
ФУНКЦИИ функция PROPN
ИНОСТРАННОГО иностранного PROPN
АГЕНТА агент PROPN


In [73]:
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

NotImplementedError: [E894] The 'noun_chunks' syntax iterator is not implemented for language 'ru'.

# 1. Ошибки токенизации

Найдите 1 любой способ сломать токенизацию на предложения функцией sentenize из библиотеки razdel.
Придумайте (или найдите на каком-то корпусе) такое предложение (или несколько предложений),
которое будет некорректно разобрано sentenize,
но при этом будет грамматически верным.

In [78]:
from razdel import sentenize as razdel_sentenize

In [88]:
text1 = 'Сентенизатор ищет заглавную букву после завершающего знака препинания. однако с точки зрения русского языка первое слово в предложении, начинающееся с маленькой буквы, не является грамматической ошибкой.'
list(razdel_sentenize(text1))

[Substring(0,
           203,
           'Сентенизатор ищет заглавную букву после завершающего знака препинания. однако с точки зрения русского языка первое слово в предложении, начинающееся с маленькой буквы, не является грамматической ошибкой.')]

In [89]:
text2 = 'Также, сентенизатор не распознаёт схожие по начертанию символы, не так ли？Это тоже не является грамматической ошибкой.'
list(razdel_sentenize(text2))

[Substring(0,
           118,
           'Также, сентенизатор не распознаёт схожие по начертанию символы, не так ли？Это тоже не является грамматической ошибкой.')]

# 2. Токенизация Mystem vs razdel.tokenize

Токенизируйте текст (не менее 10 предложений, можно взять любую статью Вики) с помощью razdel и с помощью Mystem. Найдите различия в токенизациях. Что, по вашему мнению, работает лучше на этом тексте?

In [91]:
text3 = '''Призрак бродит по Европе - призрак коммунизма. Все силы старой Европы объединились для священной травли этого призрака: папа и царь, Меттерних и Гизо, французские радикалы и немецкие полицейские.
Где та оппозиционная партия, которую ее противники, стоящие у власти, не ославили бы коммунистической? Где та оппозиционная партия, которая в свою очередь не бросала бы клеймящего обвинения в коммунизме как более передовым представителям оппозиции, так и своим реакционным противникам?
Два вывода вытекают из этого факта.
Коммунизм признается уже силой всеми европейскими силами.
Пора уже коммунистам перед всем миром открыто изложить свои взгляды, свои цели, свои стремления и сказкам о призраке коммунизма противопоставить манифест самой партии.
С этой целью в Лондоне собрались коммунисты самых различных национальностей и составили следующий "Манифест", который публикуется на английском, французском, немецком, итальянском, фламандском и датском языках.'''

list(razdel_tokenize(text3))

[Substring(0, 7, 'Призрак'),
 Substring(8, 14, 'бродит'),
 Substring(15, 17, 'по'),
 Substring(18, 24, 'Европе'),
 Substring(25, 26, '-'),
 Substring(27, 34, 'призрак'),
 Substring(35, 45, 'коммунизма'),
 Substring(45, 46, '.'),
 Substring(47, 50, 'Все'),
 Substring(51, 55, 'силы'),
 Substring(56, 62, 'старой'),
 Substring(63, 69, 'Европы'),
 Substring(70, 82, 'объединились'),
 Substring(83, 86, 'для'),
 Substring(87, 96, 'священной'),
 Substring(97, 103, 'травли'),
 Substring(104, 109, 'этого'),
 Substring(110, 118, 'призрака'),
 Substring(118, 119, ':'),
 Substring(120, 124, 'папа'),
 Substring(125, 126, 'и'),
 Substring(127, 131, 'царь'),
 Substring(131, 132, ','),
 Substring(133, 142, 'Меттерних'),
 Substring(143, 144, 'и'),
 Substring(145, 149, 'Гизо'),
 Substring(149, 150, ','),
 Substring(151, 162, 'французские'),
 Substring(163, 171, 'радикалы'),
 Substring(172, 173, 'и'),
 Substring(174, 182, 'немецкие'),
 Substring(183, 194, 'полицейские'),
 Substring(194, 195, '.'),
 Substri

In [99]:
tokens_mystem = mystem.analyze(text3)
print('\n'.join(str(i) for i in tokens_mystem[:3]), '\n')

for token in tokens_mystem:
    print(token['text'])

{'analysis': [{'lex': 'призрак', 'wt': 1, 'gr': 'S,муж=(им,ед,од|вин,ед,неод|им,ед,неод)'}], 'text': 'Призрак'}
{'text': ' '}
{'analysis': [{'lex': 'бродить', 'wt': 1, 'gr': 'V,несов,нп=непрош,ед,изъяв,3-л'}], 'text': 'бродит'} 

Призрак
 
бродит
 
по
 
Европе
 - 
призрак
 
коммунизма
. 
Все
 
силы
 
старой
 
Европы
 
объединились
 
для
 
священной
 
травли
 
этого
 
призрака
: 
папа
 
и
 
царь
, 
Меттерних
 
и
 
Гизо
, 
французские
 
радикалы
 
и
 
немецкие
 
полицейские
.


Где
 
та
 
оппозиционная
 
партия
, 
которую
 
ее
 
противники
, 
стоящие
 
у
 
власти
, 
не
 
ославили
 
бы
 
коммунистической
? 
Где
 
та
 
оппозиционная
 
партия
, 
которая
 
в
 
свою
 
очередь
 
не
 
бросала
 
бы
 
клеймящего
 
обвинения
 
в
 
коммунизме
 
как
 
более
 
передовым
 
представителям
 
оппозиции
, 
так
 
и
 
своим
 
реакционным
 
противникам
?


Два
 
вывода
 
вытекают
 
из
 
этого
 
факта
.


Коммунизм
 
признается
 
уже
 
силой
 
всеми
 
европейскими
 
силами
.


Пора
 
уже
 
коммунистам
 
перед

# 3. Лемматизация Mystem vs Pymorphy

Лемматизируйте текст с помощью mystem и pymorphy.
Найдите различия в лемматизации.
Что, по вашему мнению, работает лучше на этом тексте?

In [116]:
words_analized = [morph.parse(token['text']) for token in tokens_mystem]
# Она похожа на analyze в майстеме только возрващает список объектов Parse
# Первый в списке - самый вероятный разбор (у каждого есть score)
# Информация достается через атрибут (Parse.word - например)
# Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
# части речи или другие категории
for word in words_analized:
    if not word[0].word.isspace():
        print(word[0].word, '-', word[0].normal_form, '---', word[0], '\n')

призрак - призрак --- Parse(word='призрак', tag=OpencorporaTag('NOUN,anim,masc sing,nomn'), normal_form='призрак', score=0.75, methods_stack=((DictionaryAnalyzer(), 'призрак', 2, 0),)) 

бродит - бродить --- Parse(word='бродит', tag=OpencorporaTag('VERB,impf,intr sing,3per,pres,indc'), normal_form='бродить', score=1.0, methods_stack=((DictionaryAnalyzer(), 'бродит', 498, 5),)) 

по - по --- Parse(word='по', tag=OpencorporaTag('PREP'), normal_form='по', score=0.998387, methods_stack=((DictionaryAnalyzer(), 'по', 24, 0),)) 

европе - европа --- Parse(word='европе', tag=OpencorporaTag('NOUN,inan,femn,Sgtm,Geox sing,loct'), normal_form='европа', score=0.892086, methods_stack=((DictionaryAnalyzer(), 'европе', 36, 6),)) 

 -  -  -  --- Parse(word=' - ', tag=OpencorporaTag('PNCT'), normal_form=' - ', score=1.0, methods_stack=((PunctuationAnalyzer(score=0.9), ' - '),)) 

призрак - призрак --- Parse(word='призрак', tag=OpencorporaTag('NOUN,anim,masc sing,nomn'), normal_form='призрак', score=0.7

In [118]:
for token in tokens_mystem:
    if not token['text'].isspace():
        if 'analysis' in token:
            print(token['text'], '-', token['analysis'][0]['lex'], '---', token)
        else:
            print(token['text'], '---', token)

Призрак - призрак --- {'analysis': [{'lex': 'призрак', 'wt': 1, 'gr': 'S,муж=(им,ед,од|вин,ед,неод|им,ед,неод)'}], 'text': 'Призрак'}
бродит - бродить --- {'analysis': [{'lex': 'бродить', 'wt': 1, 'gr': 'V,несов,нп=непрош,ед,изъяв,3-л'}], 'text': 'бродит'}
по - по --- {'analysis': [{'lex': 'по', 'wt': 1, 'gr': 'PR='}], 'text': 'по'}
Европе - европа --- {'analysis': [{'lex': 'европа', 'wt': 1, 'gr': 'S,гео,жен,неод=(пр,ед|дат,ед)'}], 'text': 'Европе'}
 -  --- {'text': ' - '}
призрак - призрак --- {'analysis': [{'lex': 'призрак', 'wt': 1, 'gr': 'S,муж=(им,ед,од|вин,ед,неод|им,ед,неод)'}], 'text': 'призрак'}
коммунизма - коммунизм --- {'analysis': [{'lex': 'коммунизм', 'wt': 1, 'gr': 'S,муж,неод=род,ед'}], 'text': 'коммунизма'}
.  --- {'text': '. '}
Все - весь --- {'analysis': [{'lex': 'весь', 'wt': 0.5638720238, 'gr': 'APRO=(им,мн|вин,ед,сред|им,ед,сред|вин,мн,неод)'}], 'text': 'Все'}
силы - сила --- {'analysis': [{'lex': 'сила', 'wt': 0.9988700383, 'gr': 'S,жен,неод=(вин,мн|род,ед|им,мн